In [1]:
import os
from langchain.schema import SystemMessage
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.tools import Tool
from transformers import pipeline

os.environ["OPENAI_API_KEY"] = ''

In [2]:
def fallacy_detector(text):
    pipe = pipeline(model="fzanartu/flicc", device='cuda')
    result = pipe([text])
    return result[0].get('label')

In [3]:
system_message = SystemMessage(
    content =   """You are an expert climate analyst, who provide precise and consise responses to climate change misinformation.

                Please make sure you complete the objective above with the following rules:

                1/ Provide precise and concise replies to climate change misinformation using a structured "hamburger-style" response, including the myth, the fallacy, and the fact.
                2/ You should use the provided fallacy detector tool to identify the correct fallacy contained in the text.
                3/ Hamburger-Style Response: Your responses should follow a structured format commonly known as the "hamburger-style." This includes:
                Top Bun (Myth): Start with the misinformation related to climate change.
                Meat (Fallacy): Utilize the provided fallacy detector tool to identify the specific fallacy in the misinformation. Explain why the misinformation is incorrect, addressing the fallacious reasoning present.
                Bottom Bun (Fact): End with a clear and concise presentation of the factual information that debunks the climate change myth.

                Your task is complete once you have written all the elements of the hamburger-style response.
                """
            )

tools = [
    Tool(
        name="fallacy_detector",
        func=fallacy_detector,
        description="useful when you need to detect a fallacy"
    ),
]

agent_kwargs = {
    "system_message": system_message,
}

llm = ChatOpenAI(temperature=0.0, model="gpt-3.5-turbo")

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    agent_kwargs=agent_kwargs,
)

In [4]:
response = agent.run("So many more people die from cold than from heat in the UK.")



> Entering new AgentExecutor chain...

Invoking: `fallacy_detector` with `So many more people die from cold than from heat in the UK.`


oversimplificationTop Bun (Myth): So many more people die from cold than from heat in the UK.

Meat (Fallacy): The fallacy in this statement is oversimplification. While it is true that cold weather can have adverse health effects, it is an oversimplification to claim that more people die from cold than from heat in the UK without considering other factors and nuances.

Bottom Bun (Fact): The fact is that both cold and heat can have significant impacts on human health. While cold weather can increase the risk of respiratory and cardiovascular diseases, heatwaves can lead to heatstroke, dehydration, and exacerbation of existing health conditions. According to studies, heat-related deaths have been increasing in the UK due to the effects of climate change. It is important to address both cold and heat-related risks and take appropriate measures to pro

In [6]:
print(response)

Top Bun (Myth): So many more people die from cold than from heat in the UK.

Meat (Fallacy): The fallacy in this statement is oversimplification. While it is true that cold weather can have adverse health effects, it is an oversimplification to claim that more people die from cold than from heat in the UK without considering other factors and nuances.

Bottom Bun (Fact): The fact is that both cold and heat can have significant impacts on human health. While cold weather can increase the risk of respiratory and cardiovascular diseases, heatwaves can lead to heatstroke, dehydration, and exacerbation of existing health conditions. According to studies, heat-related deaths have been increasing in the UK due to the effects of climate change. It is important to address both cold and heat-related risks and take appropriate measures to protect public health.
